In [18]:
import os
import numpy as np
import obspy as obs
import plotly.graph_objects as go
from scipy.signal import correlate, butter, lfilter
from numpy.fft import rfft, irfft, fft, ifft

In [19]:
def sub_mid(array):
    return array - array.mean()


def taper(data):
    length = int(len(data)*5//1000)
    ox = [i/length*np.pi/2 for i in range(length)]
    sinus = np.sin(ox)/max(np.sin(ox))
    data[:length] = data[:length] * sinus
    data[(len(data)-length):] = data[(len(data)-length):] * sinus[::-1]
    return data


def sub_tap(stream):
    for i in range(len(stream)):
        stream[i].data = taper(sub_mid(stream[i].data))


def one_bit(array):
    array[array>0] = 1
    array[array<0] = -1
    return array


def whitening(array):
    spectre = rfft(array)
    new_array = irfft(spectre/np.abs(spectre))
    return new_array


def window_correlation(array1, array2, window):
    all_corr = np.zeros(window)
    iterations = len(array1)//window
    for i in range(iterations):
        all_corr += correlate(array1[i*window:(i+1)*window], array2[i*window:(i+1)*window], mode='same')
    return all_corr

In [20]:
def draw(x, y, title, xrange):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=x, 
                   y=y, 
                   showlegend=False))
    fig.update_layout(title=title,
                      template='seaborn')
    fig.update_xaxes(range=range)
    fig.show()

In [21]:
stp = obs.read('https://raw.github.com/ashimrijal/NoiseCorrelation/master/data/noise.CI.MLAC.LHZ.2004.294.2005.017.mseed')
stp += obs.read('https://raw.github.com/ashimrijal/NoiseCorrelation/master/data/noise.CI.PHL.LHZ.2004.294.2005.017.mseed')

In [22]:
minutes = 60 # Размер окна в минутах

# Параметры фильтра
lowcut = 0.1
highcut = 0.2

stn = stp.copy()
t = stn[0].stats.starttime
stn.trim(t, t + 4 * 86400-1)

sub_tap(stn)
stn.filter('bandpass', freqmin=lowcut, freqmax=highcut, zerophase=True)

8 Trace(s) in Stream:
CI.MLAC..LHZ | 2004-10-20T00:00:00.230799Z - 2004-10-21T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-21T00:00:00.230799Z - 2004-10-22T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-22T00:00:00.230799Z - 2004-10-23T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-23T00:00:00.230799Z - 2004-10-23T23:59:59.230799Z | 1.0 Hz, 86400 samples
CI.PHL..LHZ  | 2004-10-20T00:00:00.000000Z - 2004-10-21T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ  | 2004-10-21T00:00:00.000000Z - 2004-10-22T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ  | 2004-10-22T00:00:00.000000Z - 2004-10-23T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ  | 2004-10-23T00:00:00.000000Z - 2004-10-23T23:59:59.000000Z | 1.0 Hz, 86400 samples

In [23]:
fs = stn[0].stats.sampling_rate
window_size = int(minutes*60*fs)
osx = np.linspace(-window_size/(2*fs), window_size/(2*fs), window_size)

In [24]:
first = stn[0:4]
second = stn[4::]
first, second

(4 Trace(s) in Stream:
CI.MLAC..LHZ | 2004-10-20T00:00:00.230799Z - 2004-10-21T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-21T00:00:00.230799Z - 2004-10-22T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-22T00:00:00.230799Z - 2004-10-23T00:00:00.230799Z | 1.0 Hz, 86401 samples
CI.MLAC..LHZ | 2004-10-23T00:00:00.230799Z - 2004-10-23T23:59:59.230799Z | 1.0 Hz, 86400 samples,
 4 Trace(s) in Stream:
CI.PHL..LHZ | 2004-10-20T00:00:00.000000Z - 2004-10-21T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ | 2004-10-21T00:00:00.000000Z - 2004-10-22T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ | 2004-10-22T00:00:00.000000Z - 2004-10-23T00:00:00.000000Z | 1.0 Hz, 86401 samples
CI.PHL..LHZ | 2004-10-23T00:00:00.000000Z - 2004-10-23T23:59:59.000000Z | 1.0 Hz, 86400 samples)

In [25]:
big_correlation = np.zeros(window_size)

for i in range(len(first)):
    array1 = first[i].data
    array2 = second[i].data
    
    array1, array2 = whitening(array1), whitening(array2)  # Вайтенинг спектра
    array1, array2 = one_bit(array1), one_bit(array2)  # Однобитная нормализация
    
    big_correlation += window_correlation(array1, array2, window_size)

In [26]:
draw(osx, big_correlation, 'Cross-corelation', (-200, 200))

ValueError: 
    Invalid value of type 'builtins.range' received for the 'range' property of layout.xaxis
        Received value: <class 'range'>

    The 'range' property is an info array that may be specified as:

    * a list or tuple of 2 elements where:
(0) The 'range[0]' property accepts values of any type
(1) The 'range[1]' property accepts values of any type

In [33]:
f = open('american_correlation.txt')
american = []
for i in f:
    american.append(float(i))

In [36]:
len(american)

7201

In [37]:
len(big_correlation)

3600

In [48]:
f1 = np.array(american[3600-1800:3600+1800])
f2 = big_correlation
line_x = np.linspace(-len(f1)/2, len(f1)/2, len(f1))

In [49]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=line_x, 
                y=f1/max(f1), name='American colleagues',
                showlegend=True))
fig.add_trace(
    go.Scatter(x=line_x, 
                y=f2/max(f2), name='My',
                showlegend=True))
fig.update_layout(title='Comparison of results', 
                    template='seaborn')
fig.show()